In [3]:
# third-party
import pandas as pd
import numpy as np
import scipy
import pickle

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from fr_performance import get_fr_detection_performance
from delay_performance import normality_test, plot_inspiration_vs_expiration
from constants import CATEGORICAL_PALETTE


In [10]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]


### Activity-specific only FR analysis

In [16]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [17]:
performance_all_participants = get_fr_detection_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,SNB,MAG,1.02,0.97,0.99,-0.07 $\pm$ 0.43,-1 %
1,SNB,PZT,1.14,0.87,0.98,-0.39 $\pm$ 0.58,-9 %
2,SGB,MAG,1.00,0.94,0.94,-0.23 $\pm$ 0.95,-2 %
3,SGB,PZT,1.62,0.60,0.98,-0.87 $\pm$ 1.68,-8 %
4,MIXB,MAG,0.99,0.99,0.98,-0.07 $\pm$ 0.4,-1 %
5,MIXB,PZT,1.12,0.84,0.94,-0.47 $\pm$ 0.86,-9 %
6,STNB,MAG,1.05,0.95,0.99,-0.08 $\pm$ 0.37,-1 %
7,STNB,PZT,1.05,0.93,0.98,-0.31 $\pm$ 0.86,-6 %
8,MCH,MAG,1.17,0.85,0.99,-0.11 $\pm$ 0.4,-3 %
9,MCH,PZT,1.59,0.61,0.96,-0.09 $\pm$ 0.69,-2 %


In [5]:
mag = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].loc[:,["Ratio", "Precision", "Recall"]].reset_index(drop=True) 
pzt = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].loc[:,["Ratio", "Precision", "Recall"]].reset_index(drop=True)

a = mag / pzt 
a["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
a

,Ratio,Precision,Recall,Activity
0,0.894737,1.114943,1.010204,SNB
1,0.617284,1.566667,0.959184,SGB
2,0.883929,1.178571,1.042553,MIXB
3,1.000000,1.021505,1.010204,STNB
4,0.735849,1.393443,1.031250,MCH
5,0.770833,1.308824,1.010204,SQT
6,0.856000,1.151899,0.989899,AAL
7,0.808000,1.240506,1.000000,AAR
8,0.764706,1.338028,1.010309,ALL
9,0.801587,1.376812,1.103448,ALR


In [6]:
mag["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
mag

,Ratio,Precision,Recall,Activity
0,1.02,0.97,0.99,SNB
1,1.00,0.94,0.94,SGB
2,0.99,0.99,0.98,MIXB
3,1.05,0.95,0.99,STNB
4,1.17,0.85,0.99,MCH
5,1.11,0.89,0.99,SQT
6,1.07,0.91,0.98,AAL
7,1.01,0.98,0.99,AAR
8,1.04,0.95,0.98,ALL
9,1.01,0.95,0.96,ALR


### Participant-specific only FR analysis

In [7]:
# Transform overview from participant specific into all participants
overview_all_activities = transform_overview_on_target(overview, target="ID")
overview_all_activities.keys()

dict_keys(['7OYX', 'NO15', 'G8B7', 'EPE2', 'HAK8', '1BST', '83J1', 'QMQ7', '9TUL', 'FTD7', 'Y6O3', '2QWT', 'F9AF', 'P4W9', 'W8Z9', 'D4GQ'])

In [8]:
get_fr_detection_performance(overview_all_activities, target="ID")

,ID,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,7OYX,MAG,1.09,0.91,1.00,-0.07 $\pm$ 0.38,-1 %
1,7OYX,PZT,1.85,0.53,0.99,-0.22 $\pm$ 0.88,-5 %
2,NO15,MAG,1.13,0.88,0.99,-0.12 $\pm$ 0.39,-2 %
3,NO15,PZT,1.52,0.65,0.99,-0.4 $\pm$ 0.64,-9 %
4,G8B7,MAG,1.14,0.83,0.94,-0.22 $\pm$ 0.56,-6 %
5,G8B7,PZT,1.06,0.93,0.98,-0.36 $\pm$ 0.43,-10 %
6,EPE2,MAG,1.00,0.97,0.97,0.17 $\pm$ 0.48,3 %
7,EPE2,PZT,1.36,0.72,0.98,-0.38 $\pm$ 0.85,-8 %
8,HAK8,MAG,1.11,0.89,0.98,-0.1 $\pm$ 0.36,-2 %
9,HAK8,PZT,1.44,0.66,0.95,0.15 $\pm$ 0.7,4 %


### Overall FR Analysis

In [9]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['ScientISST', 'BIOPAC', 'BITalino'])

In [10]:
get_fr_detection_performance(overview_all, target=None)

,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,MAG,1.10,0.89,0.98,-0.07 $\pm$ 0.44,-1 %
1,PZT,1.35,0.71,0.96,-0.16 $\pm$ 0.76,-3 %


### Overall delay analysis

In [11]:
delays_i_all_scientisst, delays_e_all_scientisst = [], []
delays_i_all_bitalino, delays_e_all_bitalino = [], []

for activity in overview_all_participants.keys():

    delays_i_all_scientisst += overview_all_participants[activity]["ScientISST"]["delay_i"] 
    delays_e_all_scientisst += overview_all_participants[activity]["ScientISST"]["delay_e"]

    delays_i_all_bitalino += overview_all_participants[activity]["BITalino"]["delay_i"]
    delays_e_all_bitalino += overview_all_participants[activity]["BITalino"]["delay_e"]

In [12]:
print(f"min delay scientisst {min(delays_i_all_scientisst + delays_e_all_scientisst, key=abs)}")
print(f"min delay bitalino {min(delays_i_all_bitalino + delays_e_all_bitalino, key=abs)}")

print(f"max delay scientisst {max(delays_i_all_scientisst + delays_e_all_scientisst, key=abs)}")
print(f"max delay bitalino {max(delays_i_all_bitalino + delays_e_all_bitalino, key=abs)}")

print(f"mean delay scientisst {round(np.mean(delays_i_all_scientisst + delays_e_all_scientisst), 2)} +/- {np.round(np.std(delays_i_all_scientisst + delays_e_all_scientisst), 2)}")
print(f"mean delay bitalino {np.round(np.mean(delays_i_all_bitalino + delays_e_all_bitalino), 2)} +/- {np.round(np.std(delays_i_all_bitalino + delays_e_all_bitalino), 2)}")

min delay scientisst -0.0
min delay bitalino -0.0
max delay scientisst -2.93
max delay bitalino -4.92
mean delay scientisst -0.07 +/- 0.44
mean delay bitalino -0.16 +/- 0.76


In [13]:
normality_test(delays_i_all_scientisst + delays_e_all_scientisst, sensor="MAG", type="inspiration+expiration")
normality_test(delays_i_all_bitalino + delays_e_all_bitalino, sensor="PZT", type="inspiration+expiration")

### Event-specific delay analysis

In [14]:
normality_test(delays_i_all_scientisst, sensor="MAG", type="inspiration")
normality_test(delays_e_all_scientisst, sensor="MAG", type="expiration")
normality_test(delays_i_all_bitalino, sensor="PZT", type="inspiration", categorical_palette=CATEGORICAL_PALETTE[2:])
normality_test(delays_e_all_bitalino, sensor="PZT", type="expiration", categorical_palette=CATEGORICAL_PALETTE[2:])

In [15]:
delays_df = pd.DataFrame(columns=["Sensor", "Type", "Delay"])
delays_df["Delay"] = delays_i_all_scientisst + delays_e_all_scientisst + delays_i_all_bitalino + delays_e_all_bitalino
delays_df["Sensor"] = ["MAG"] * len(delays_i_all_scientisst + delays_e_all_scientisst) + ["PZT"] * len(delays_i_all_bitalino + delays_e_all_bitalino)
delays_df["Type"] = ["Inhalation"] * len(delays_i_all_scientisst) + ["Exhalation"] * len(delays_e_all_scientisst) + ["Inhalation"] * len(delays_i_all_bitalino) + ["Exhalation"] * len(delays_e_all_bitalino)
delays_df


,Sensor,Type,Delay
0,MAG,Inhalation,-0.22
1,MAG,Inhalation,-0.05
2,MAG,Inhalation,-0.11
3,MAG,Inhalation,0.01
4,MAG,Inhalation,0.12
...,...,...,...
13960,PZT,Exhalation,-0.06
13961,PZT,Exhalation,-1.35
13962,PZT,Exhalation,-0.87
13963,PZT,Exhalation,-0.28


In [16]:
plot_inspiration_vs_expiration(delays_df)

#### Statistical testing: comparing inhalation vs expiration

In [17]:
print(scipy.stats.mannwhitneyu(delays_i_all_scientisst, delays_e_all_scientisst))
print(scipy.stats.mannwhitneyu(delays_i_all_bitalino, delays_e_all_bitalino))

MannwhitneyuResult(statistic=6072700.0, pvalue=0.09286967929195358)
MannwhitneyuResult(statistic=5942065.5, pvalue=0.7030550024874435)


In [18]:
print(len(delays_i_all_scientisst + delays_e_all_scientisst))
print(len(delays_i_all_bitalino + delays_e_all_bitalino))

7052
6913


### Correlation between strap-band perimeter and FR performance

In [19]:
strap_thorax_ratio_dict = {
    '7OYX': 0.94,
    'NO15': 0.99,
    'G8B7': 0.96,
    'EPE2': 0.93,
    'HAK8': 0.94,
    '1BST': 0.98,
    '83J1': 0.97,
    'QMQ7': 0.96,
    '9TUL': 0.9,
    'FTD7': 0.9,
    'Y6O3': 0.92,
    '2QWT': 0.85,
    'F9AF': 0.87,
    'P4W9': 0.96,
    'W8Z9': 0.94,
    'D4GQ': 0.92
}

In [20]:
strap_thorax_performance_df = get_fr_detection_performance(overview, target="both")
strap_thorax_performance_df = strap_thorax_performance_df[strap_thorax_performance_df["Sensor"] == "MAG"]
strap_thorax_performance_df['strap-to-thorax ratio'] = strap_thorax_performance_df["ID"].apply(lambda id: strap_thorax_ratio_dict[id])
strap_thorax_performance_df


,ID,Activity,Sensor,Ratio,Precision,Recall,strap-to-thorax ratio
0,7OYX,SNB,MAG,1.00,1.00,1.00,0.94
2,7OYX,SGB,MAG,1.00,1.00,1.00,0.94
4,7OYX,MIXB,MAG,1.00,1.00,1.00,0.94
6,7OYX,STNB,MAG,1.00,1.00,1.00,0.94
8,7OYX,MCH,MAG,0.95,1.00,0.95,0.94
...,...,...,...,...,...,...,...
468,D4GQ,ALR,MAG,0.58,1.00,0.58,0.92
470,D4GQ,UAL,MAG,2.55,0.39,1.00,0.92
472,D4GQ,UAR,MAG,1.05,0.91,0.95,0.92
474,D4GQ,SE,MAG,1.33,0.75,1.00,0.92


In [21]:
scipy.stats.spearmanr(strap_thorax_performance_df["Ratio"].values, strap_thorax_performance_df["strap-to-thorax ratio"].values)

SignificanceResult(statistic=0.091521055630493, pvalue=0.15841421153751592)

### Analysis on alternative positioning

In [27]:
overview_middle_all_participants = transform_overview_on_target(overview_middle, target="Activity")
performance_middle_all_participants = get_fr_detection_performance(overview_middle_all_participants, target="Activity")
performance_middle_all_participants = performance_middle_all_participants[performance_middle_all_participants["Sensor"]=="MAG"]
performance_middle_all_participants

,Activity,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,SNBm,MAG,1.00,0.98,0.98,-0.02 $\pm$ 0.39,0 %
2,UALm,MAG,1.26,0.78,0.98,0.16 $\pm$ 0.55,5 %
4,UARm,MAG,1.36,0.72,0.98,0.22 $\pm$ 0.61,5 %


In [34]:
performance_middle_vs_left = pd.DataFrame(columns=performance_middle_all_participants.columns)

for activity in performance_middle_all_participants["Activity"].unique():
    new_entry = performance_all_participants[(performance_all_participants["Activity"]==activity[:-1]) & (performance_all_participants["Sensor"]=="MAG")]
    performance_middle_vs_left = pd.concat([performance_middle_vs_left, new_entry], ignore_index=True)

    new_entry = performance_middle_all_participants[performance_middle_all_participants["Activity"]==activity]
    performance_middle_vs_left = pd.concat([performance_middle_vs_left, new_entry], ignore_index=True)

performance_middle_vs_left.drop(columns=["Sensor", "Mean absolute delay $\pm$ SD", "Adjusted delay"], inplace=True)
performance_middle_vs_left

,Activity,Ratio,Precision,Recall
0,SNB,1.02,0.97,0.99
1,SNBm,1.00,0.98,0.98
2,UAL,1.17,0.85,0.99
3,UALm,1.26,0.78,0.98
4,UAR,1.08,0.93,1.00
5,UARm,1.36,0.72,0.98


In [22]:
delays_middle_i_all_scientisst, delays_middle_e_all_scientisst = [], []
delays_middle_i_all_bitalino, delays_middle_e_all_bitalino = [], []

for activity in overview_middle_all_participants.keys():

    delays_middle_i_all_scientisst += overview_middle_all_participants[activity]["ScientISST"]["delay_i"] 
    delays_middle_e_all_scientisst += overview_middle_all_participants[activity]["ScientISST"]["delay_e"]

    delays_middle_i_all_bitalino += overview_middle_all_participants[activity]["BITalino"]["delay_i"]
    delays_middle_e_all_bitalino += overview_middle_all_participants[activity]["BITalino"]["delay_e"]

print(f"min delay scientisst {min(delays_middle_i_all_scientisst + delays_middle_e_all_scientisst, key=abs)}")
print(f"min delay bitalino {min(delays_middle_i_all_bitalino + delays_middle_e_all_bitalino, key=abs)}")

print(f"max delay scientisst {max(delays_middle_i_all_scientisst + delays_middle_e_all_scientisst, key=abs)}")
print(f"max delay bitalino {max(delays_middle_i_all_bitalino + delays_middle_e_all_bitalino, key=abs)}")

print(f"mean delay scientisst {round(np.mean(delays_middle_i_all_scientisst + delays_middle_e_all_scientisst), 2)} +/- {np.round(np.std(delays_middle_i_all_scientisst + delays_middle_e_all_scientisst), 2)}")
print(f"mean delay bitalino {np.round(np.mean(delays_middle_i_all_bitalino + delays_middle_e_all_bitalino), 2)} +/- {np.round(np.std(delays_middle_i_all_bitalino + delays_middle_e_all_bitalino), 2)}")

min delay scientisst -0.0
min delay bitalino -0.0
max delay scientisst -3.22
max delay bitalino 4.68
mean delay scientisst 0.08 +/- 0.5
mean delay bitalino -0.47 +/- 0.86
